In [2]:
import face_recognition
import cv2
import MySQLdb
import datetime
import time
import skvideo.io
import numpy as np

In [1]:

def face_detection():
    
    """               
    Function impelmets the face detection using face_recognisation library.
    It will detect the all known faces in the image_path_list and display the person name
    in case of the match else it will show Unknown keyword.
    """
    #start capture the frame using 
    video_capture = cv2.VideoCapture(0)
    image_path_list=["/home/awanish/Desktop/University/Resume&CoverLetter/DSC_0148.jpg","/home/awanish/Desktop/University/Resume&CoverLetter/kam.jpg"]
    
    # Define the codec and create VideoWriter object.The output is stored in 'outpy.avi' file.
    # Define the fps to be equal to 10. Also frame size is passed.
    fourcc = cv2.cv.CV_FOURCC(*'FMP4') 
    #fourcc = cv2.VideoWriter_fourcc(*'XVID')
    out = cv2.VideoWriter('/home/awanish/Videos/output.avi',-1, 20.0, (int(video_capture.get(3)), int(video_capture.get(4))))


    awanish_image = face_recognition.load_image_file(image_path_list[0])
    awanish_face_encoding = face_recognition.face_encodings(awanish_image)[0]

    xyz_image = face_recognition.load_image_file(image_path_list[1])
    xyz_face_encoding = face_recognition.face_encodings(xyz_image)[0]


    known_encodings = [

        awanish_face_encoding,
        xyz_face_encoding
    ]
    known_names = [

        "Kumar",
        "Sam"
    ]

    face_locations = []
    face_encodings = []
    face_names = []
    process_this_frame = True
    flag = ""
    recognized = ""

    while True: #true (video_capture.isOpened())
        # Grab a single frame of video
        ret, frame = video_capture.read()


        # Resize frame of video to 1/4 size for faster face recognition processing
        small_frame = cv2.resize(frame, (0, 0), fx=1.0, fy=1.0)

        # Convert the image from BGR color (which OpenCV uses) to RGB color (which face_recognition uses)
        rgb_small_frame = small_frame[:, :, ::-1]

        out.write(frame)

        # Only process every other frame of video to save time
        if process_this_frame:
            # Find all the faces and face encodings in the current frame of video
            face_locations = face_recognition.face_locations(rgb_small_frame)
            face_encodings = face_recognition.face_encodings(rgb_small_frame, face_locations)

            face_names = []
            for face_encoding in face_encodings:
                # See if the face is a match for the known face(s)
                matches = face_recognition.compare_faces(known_encodings, face_encoding, tolerance=0.5)
                name = "Don't know"



                # If a match was found in known_face_encodings, just use the first one.
                if True in matches:

                    first_match_index = matches.index(True)
                    #print(first_match_index)
                    name = known_names[first_match_index]

                face_names.append(name)
                recognized = face_names[0]

        process_this_frame = not process_this_frame


        if not flag == recognized and not recognized == "Unknown":

            ts = time.time()
            timestamp = datetime.datetime.fromtimestamp(ts).strftime('%Y-%m-%d %H:%M:%S')
            flag = recognized


        # Display the results
        for (top, right, bottom, left), name in zip(face_locations, face_names):
            # Scale back up face locations since the frame we detected in was scaled to 1/4 size
            top *=1
            right *= 1
            bottom *= 1
            left *=1

            # Draw a box around the face
            cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 0), 2)

            # Draw a label with a name below the face
            cv2.rectangle(frame, (left - 110, bottom + 70), (right + 110, bottom), (0, 0, 0))
            font = cv2.FONT_HERSHEY_DUPLEX
            cv2.putText(frame, name, (left, bottom + 30), font, 1.0, (255, 255, 255), 1)
            #print("name={}".format(name))
            ts = time.time()
            timestamp = datetime.datetime.fromtimestamp(ts).strftime('%Y-%m-%d %H:%M:%S')
            cv2.putText(frame, timestamp,(left - 110, bottom + 60), font, 1.0, (255, 255, 255), 1)




            # write the flipped frame
            #frame = cv2.flip(frame,180)
            #print(frame.shape) # check to see frame size
            out.write(frame)


        # Display the resulting image
        cv2.imshow('Video', frame)


        # Hit 'q' on the keyboard to quit!
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # Release handle to the webcam
    video_capture.release()
    cv2.destroyAllWindows()

In [ ]:
face_detection()